Copyright **`(c)`** 2024 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free under certain conditions — see the [`license`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [2]:
import numpy as np
from icecream import ic

def true_f(x: np.ndarray) -> np.ndarray:
    return x[0] + np.sin(x[1]) / 5

TEST_SIZE = 10_000
TRAIN_SIZE = 1000

x_validation = np.vstack(
    [
        np.random.random_sample(size=TEST_SIZE) * 2 * np.pi - np.pi,
        np.random.random_sample(size=TEST_SIZE) * 2 - 1,
    ]
)
y_validation = true_f(x_validation)
train_indexes = np.random.choice(TEST_SIZE, size=TRAIN_SIZE, replace=False)
x_train = x_validation[:, train_indexes]
y_train = y_validation[train_indexes]
assert np.all(y_train == true_f(x_train)), "D'ho"

np.savez('problem_0.npz', x=x_train, y=y_train)




In [4]:
import random
# model 1

class Node:
    def __init__(self, value, left=None, right=None):
        self.value = value
        self.left = left
        self.right = right

    def copy(self):
        return Node(self.value, self.left.copy() if self.left else None, self.right.copy() if self.right else None)

    def get_subtrees(self):
        subtrees = []
        if self.left:
            subtrees.append(self.left)
            subtrees.extend(self.left.get_subtrees())
        if self.right:
            subtrees.append(self.right)
            subtrees.extend(self.right.get_subtrees())
        return subtrees

    def swap(self, other):
        self.value, other.value = other.value, self.value
        self.left, other.left = other.left, self.left
        self.right, other.right = other.right, self.right

    def __str__(self):
        if self.left and self.right:
            return f"({self.left} {self.value} {self.right})"
        return self.value

class GeneticProgramming:
    def __init__(self, population_size: int = 100, generations: int = 50):
        self.population_size = population_size
        self.generations = generations
        self.best_program = None
        self.operators = ['+', '-', '*', '/']
        
    def _generate_random_program(self) -> Node:
        if random.random() < 0.5:
            return Node(random.choice(['x[0]', 'x[1]']))
        
        op = random.choice(self.operators)
        left = self._generate_random_program()
        right = self._generate_random_program()
        return Node(op, left, right)

    def _evaluate_fitness(self, program: Node, X: np.ndarray, y: np.ndarray) -> float:
        try:
            func = self.get_function_from_tree(program)
            predictions = func(X)
            mse = np.mean(np.square(predictions - y))
            return 1 / (1 + mse)
        except:
            return 0.0

    def _crossover(self, parent1: Node, parent2: Node) -> Node:
        new_parent1 = parent1.copy()
        new_parent2 = parent2.copy()
        
        nodes1 = new_parent1.get_subtrees()
        nodes2 = new_parent2.get_subtrees()
        
        if len(nodes1) > 0 and len(nodes2) > 0:
            node1 = random.choice(nodes1)
            node2 = random.choice(nodes2)
            node1.swap(node2)
            
        return new_parent1

    def _mutate(self, program: Node) -> Node:
        if random.random() < 0.3:
            nodes = program.get_subtrees()
            if nodes:
                node = random.choice(nodes)
                if random.random() < 0.5:
                    node.value = random.choice(self.operators)
                else:
                    node.value = random.choice(['x[0]', 'x[1]'])
        return program

    def train(self, X: np.ndarray, y: np.ndarray) -> None:
        population = [self._generate_random_program() for _ in range(self.population_size)]
        
        for generation in range(self.generations):
            fitness_scores = [(p, self._evaluate_fitness(p, X, y)) for p in population]
            fitness_scores.sort(key=lambda x: x[1], reverse=True)
            
            if self.best_program is None or fitness_scores[0][1] > self._evaluate_fitness(self.best_program, X, y):
                self.best_program = fitness_scores[0][0].copy()
            
            new_population = []
            elite_size = self.population_size // 10
            
            new_population.extend([p[0].copy() for p in fitness_scores[:elite_size]])
            
            while len(new_population) < self.population_size:
                parent1 = random.choice(fitness_scores[:self.population_size//2])[0]
                parent2 = random.choice(fitness_scores[:self.population_size//2])[0]
                child = self._crossover(parent1, parent2)
                child = self._mutate(child)
                new_population.append(child)
            
            population = new_population

    def get_math_formula(self) -> str:
        if self.best_program is None:
            return "No program trained yet"
        return str(self.best_program)

    def get_function_from_tree(self, node: Node):
        formula = str(node)
        return lambda x: eval(formula)

    def get_function_math_formula(self):
        if self.best_program is None:
            raise ValueError("No program trained yet")
        return self.get_function_from_tree(self.best_program)


In [18]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import random
import operator

class Node:
    """
    Represents a node in the program tree.

    Attributes:
        value: The value of the node (operator, function, variable, or constant).
        left: The left child node.
        right: The right child node.
        arity: The number of arguments the node's operation takes (0, 1, or 2).
    """
    def __init__(self, value, left=None, right=None):
        self.value = value
        self.left = left
        self.right = right
        if self.value in SymbolicRegressor.operators:
          if self.value == '**':
            self.arity = 2
          elif self.value in ['+', '-', '*', '/']:
            self.arity = 2
          else:
            self.arity = 1
        elif self.value in SymbolicRegressor.functions:
          self.arity = 1
        else:
          self.arity = 0

    def evaluate(self, X):
        """
        Evaluates the node recursively for a given input.

        Args:
            X: Input data (numpy array).

        Returns:
            The result of the evaluation.
        """
        if self.arity == 2:
          if self.value == '+':
            return self.left.evaluate(X) + self.right.evaluate(X)
          elif self.value == '-':
            return self.left.evaluate(X) - self.right.evaluate(X)
          elif self.value == '*':
            return self.left.evaluate(X) * self.right.evaluate(X)
          elif self.value == '/':
            # Avoid division by zero
            divisor = self.right.evaluate(X)
            if isinstance(divisor, np.ndarray):
                divisor = np.where(divisor == 0, 1e-6, divisor)  # Replace zeros with a small value
            elif divisor == 0:
                divisor = 1e-6
            return self.left.evaluate(X) / divisor
          elif self.value == '**':
            return np.power(self.left.evaluate(X), self.right.evaluate(X))

        elif self.arity == 1:
          return self.value(self.left.evaluate(X))

        elif isinstance(self.value, int):
          if self.value < X.shape[1]:
            return X[:, self.value]
        #   else:
            # raise IndexError(f"Index {self.value} is out of bounds for axis 1 with size {X.shape[1]}")
        else:  # constant
          return [self.value]

    def get_formula(self):
        """
        Returns the mathematical formula represented by the node (subtree).
        """
        if self.arity == 2:
            return f"({self.left.get_formula()} {self.value} {self.right.get_formula()})"
        elif self.arity == 1:
            return f"{self.value.__name__}({self.left.get_formula()})"
        elif isinstance(self.value, int):
          return f"x{self.value+1}"
        else:
            return str(self.value)
    
    def get_depth(self):
      """Calculates the depth of the tree rooted at this node."""
      if self.arity == 0:
          return 1
      elif self.arity == 1:
          return 1 + self.left.get_depth()
      else:
          return 1 + max(self.left.get_depth(), self.right.get_depth())

    def copy(self):
        """Creates a deep copy of the node and its subtree."""
        left_copy = self.left.copy() if self.left else None
        right_copy = self.right.copy() if self.right else None
        return Node(self.value, left_copy, right_copy)
    
    def get_nodes_at_depth(self, target_depth, current_depth=0, nodes=None):
      """
      Collects all nodes at a specific depth in the tree.

      Args:
          target_depth: The desired depth.
          current_depth: The current depth during recursion.
          nodes: A list to store the nodes found at the target depth.

      Returns:
          A list of nodes at the target depth.
      """
      if nodes is None:
          nodes = []

      if current_depth == target_depth:
          nodes.append(self)
      elif current_depth < target_depth:
          if self.left:
              self.left.get_nodes_at_depth(target_depth, current_depth + 1, nodes)
          if self.right:
              self.right.get_nodes_at_depth(target_depth, current_depth + 1, nodes)
      return nodes
    
    def replace_subtree(self, old_subtree, new_subtree):
      """
      Replaces a subtree with another subtree within this node's subtree.

      Args:
          old_subtree: The subtree to be replaced.
          new_subtree: The subtree to replace with.
      """
      if self == old_subtree:
          self.value = new_subtree.value
          self.left = new_subtree.left
          self.right = new_subtree.right
          self.arity = new_subtree.arity
          return

      if self.left:
          if self.left == old_subtree:
              self.left = new_subtree
              return
          else:
              self.left.replace_subtree(old_subtree, new_subtree)
      if self.right:
          if self.right == old_subtree:
              self.right = new_subtree
              return
          else:
              self.right.replace_subtree(old_subtree, new_subtree)
          

class SymbolicRegressor:
    """
    A genetic programming-based symbolic regressor.

    Attributes:
        population_size: The size of the population.
        generations: The number of generations.
        operators: A list of allowed operators (+, -, *, /, **).
        functions: A list of allowed functions (e.g., np.sin, np.cos).
        const_range: The range for generating random constants.
        init_depth: The initial depth range for generating trees.
        crossover_rate: The probability of crossover.
        mutation_rate: The probability of mutation.
        p_terminal: The probability of selecting a terminal node during initialization.
        metric: The evaluation metric ('mse').
    """
    operators = ['+', '-', '*', '/', '**']
    functions = [np.sin, np.cos, np.exp]

    def __init__(self, population_size=100, generations=20, operators=['+', '-', '*', '/', '**'], functions=[np.sin, np.cos, np.exp], const_range=(-1.0, 1.0),
                 init_depth=(2, 6), crossover_rate=0.7, mutation_rate=0.1, p_terminal=0.5, metric='mse'):
        self.population_size = population_size
        self.generations = generations
        self.operators = operators
        self.functions = functions
        self.const_range = const_range
        self.init_depth = init_depth
        self.crossover_rate = crossover_rate
        self.mutation_rate = mutation_rate
        self.p_terminal = p_terminal
        self.metric = metric
        self.best_program = None
        self.best_fitness = float('inf')

    def _generate_random_tree(self, max_depth, X_train):
        """Generates a random program tree."""
        if max_depth == 0 or (max_depth > 0 and random.random() < self.p_terminal):
            # Terminal node (variable or constant)
            if random.random() < 0.5:
                return Node(random.randint(0, X_train.shape[1]-1)) #variable
            else:
                return Node(random.uniform(*self.const_range)) #constant
        else:
            # Non-terminal node (operator or function)
            if random.random() < len(self.operators) / (len(self.operators) + len(self.functions)):
                op = random.choice(self.operators)
                left = self._generate_random_tree(max_depth - 1, X_train)
                if op == '**':
                  right = Node(random.randint(0, 3)) # Exponentiation with small integers
                else:
                  right = self._generate_random_tree(max_depth - 1, X_train)
                return Node(op, left, right)
            else:
                func = random.choice(self.functions)
                return Node(func, self._generate_random_tree(max_depth - 1, X_train))

    def _calculate_fitness(self, program, X, y):
        """Calculates the fitness of a program."""
        try:
            y_pred = program.evaluate(X)
            if self.metric == 'mse':
                # Handle cases where y_pred might have invalid values
                if np.any(np.isnan(y_pred)) or np.any(np.isinf(y_pred)) :
                    print("Invalid prediction during evaluation")
                    return float('inf')
                else:
                    # print("y_pred", y_pred)
                    # print("y", y)
                    # print("shapes", len(y_pred), len(y.shape))
                    return mean_squared_error(y, y_pred)
        except (ValueError, TypeError, ZeroDivisionError, OverflowError, MemoryError) as err:
            # print("Error during evaluation:", type(err).__name__)
            # print("Error during evaluation")
            return float('inf')  # Penalize programs that raise errors

    def _crossover(self, parent1, parent2):
      """
      Performs crossover between two parent trees.

      Args:
          parent1: The first parent node.
          parent2: The second parent node.

      Returns:
          Two offspring nodes.
      """
      if parent1.arity == 0 or parent2.arity == 0:
        return parent1.copy(), parent2.copy()

      # 1. Choose a random crossover point in each parent
      crossover_point1 = random.choice(parent1.get_nodes_at_depth(random.randint(1, parent1.get_depth()-1))) if parent1.get_depth() > 1 else parent1
      crossover_point2 = random.choice(parent2.get_nodes_at_depth(random.randint(1, parent2.get_depth()-1))) if parent2.get_depth() > 1 else parent2

      # 2. Create copies of the parents to avoid modifying them directly
      offspring1 = parent1.copy()
      offspring2 = parent2.copy()
      
      offspring1.replace_subtree(crossover_point1, crossover_point2.copy())
      offspring2.replace_subtree(crossover_point2, crossover_point1.copy())
      
      return offspring1, offspring2

    def _mutation(self, program, X_train):
      """
      Performs mutation on a program tree.

      Args:
          program: The program node to mutate.

      Returns:
          The mutated program node.
      """
      mutated_program = program.copy()
      mutation_point = random.choice(mutated_program.get_nodes_at_depth(random.randint(0, mutated_program.get_depth()-1)))

      if mutation_point.arity == 0:
        # Mutate a terminal node
        if isinstance(mutation_point.value, int): #variable
          mutation_point.value = random.randint(0,1)
        else: #constant
          mutation_point.value = random.uniform(*self.const_range)
      
      elif mutation_point.arity == 1:
        # Mutate a function
        mutation_point.value = random.choice(self.functions)

      else:
        # Mutate an operator
        mutation_point.value = random.choice(self.operators)
        if mutation_point.value == "**" and random.random() < 0.7:
          mutation_point.right = Node(random.randint(0, 3))

      return mutated_program
    
    def _prune(self, program, X, y):
      """
      Prunes a program tree to simplify it while trying to maintain performance.

      Args:
          program: The program tree to prune.
          X: The input features.
          y: The target values.

      Returns:
          The pruned program tree.
      """
      original_fitness = self._calculate_fitness(program, X, y)
      best_pruned_program = program.copy()
      best_pruned_fitness = original_fitness
      
      for depth in range(program.get_depth() - 1, 0, -1):
          nodes_at_depth = program.get_nodes_at_depth(depth)
          for node in nodes_at_depth:
              if node.arity > 0:  # Only consider non-terminal nodes for pruning
                  
                  # Try replacing with a constant
                  pruned_program_constant = program.copy()
                  pruned_program_constant.replace_subtree(node, Node(random.uniform(*self.const_range)))
                  pruned_fitness_constant = self._calculate_fitness(pruned_program_constant, X, y)
                  
                  if pruned_fitness_constant <= best_pruned_fitness:
                      best_pruned_fitness = pruned_fitness_constant
                      best_pruned_program = pruned_program_constant
                  
                  # Try replacing with a variable
                  for var_index in range(X.shape[1]):
                      pruned_program_variable = program.copy()
                      pruned_program_variable.replace_subtree(node, Node(var_index))
                      pruned_fitness_variable = self._calculate_fitness(pruned_program_variable, X, y)
                      
                      if pruned_fitness_variable <= best_pruned_fitness:
                          best_pruned_fitness = pruned_fitness_variable
                          best_pruned_program = pruned_program_variable

      if best_pruned_fitness <= original_fitness * 1.1: # allow a small error
        return best_pruned_program
      else:
        return program

    def train(self, X_train, y_train):
        """Trains the symbolic regressor."""
        # 1. Initialize the population
        population = [self._generate_random_tree(random.randint(*self.init_depth), X_train) for _ in range(self.population_size)]

        for generation in range(self.generations):
            # 2. Evaluate fitness
            fitness_scores = [self._calculate_fitness(program, X_train, y_train) for program in population]
            # print(f"Generation {generation+1}, Best Fitness: {(population[0].get_formula())}")
            # 3. Store best program
            for i, fitness in enumerate(fitness_scores):
                if fitness < self.best_fitness:
                    self.best_fitness = fitness
                    self.best_program = population[i].copy()

            # 4. Selection (tournament selection)
            selected_indices = []
            for _ in range(self.population_size):
                tournament = random.sample(range(self.population_size), 2) #tournament of size 2
                winner = min(tournament, key=lambda i: fitness_scores[i])
                selected_indices.append(winner)

            # 5. Crossover and Mutation
            new_population = []
            for i in range(0, self.population_size, 2):
                parent1 = population[selected_indices[i]]
                parent2 = population[selected_indices[i+1]]

                if random.random() < self.crossover_rate:
                    offspring1, offspring2 = self._crossover(parent1, parent2)
                else:
                    offspring1, offspring2 = parent1.copy(), parent2.copy()

                if random.random() < self.mutation_rate:
                    offspring1 = self._mutation(offspring1, X_train)
                if random.random() < self.mutation_rate:
                    offspring2 = self._mutation(offspring2, X_train)

                new_population.extend([offspring1, offspring2])

            population = new_population

            # 6. Pruning (optional, you can adjust the frequency)
            # if generation % 5 == 0:
            #   for i in range(len(population)):
            #     population[i] = self._prune(population[i], X_train, y_train)
              
            print(f"Generation {generation+1}, Best Fitness: {self.best_fitness:.4f}, Best Program Depth: {100 if self.best_program is None else self.best_program.get_depth()}")

    def get_math_formula(self):
        """Returns the best program as a mathematical formula string."""
        if self.best_program is not None:
            return self.best_program.get_formula()
        else:
            return "No program trained yet."

    def get_function_math_formula(self):
      """Returns the best program as a callable function."""
      if self.best_program is not None:
          return lambda X: self.best_program.evaluate(X)
      else:
          return "No program trained yet."

# Example usage with custom functions and operators:
X = np.random.rand(100, 3)  # 3 input features
y = np.sin(X[:, 0]) + X[:, 1] ** 2 - 0.5 * X[:, 2]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create and train the symbolic regressor
regressor = SymbolicRegressor(
    population_size=2000,
    generations=20,
    operators=['+', '-', '*', '/', '**'],
    functions=[np.sin, np.cos, np.exp],
    const_range=(-2.0, 2.0),
    init_depth=(2, 5),
    crossover_rate=0.7,
    mutation_rate=0.2,
    p_terminal=0.6,
    metric='mse'
)

regressor.train(X_train, y_train)

# Get the best program as a mathematical formula
best_formula = regressor.get_math_formula()
print(f"Best program (math formula): {best_formula}")

# Get the best program as a callable function
best_function = regressor.get_function_math_formula()

# print("Best function:", best_function.)
# Make predictions on the test set
y_pred = best_function(X_test)

# Evaluate the model

mse = mean_squared_error(y_test, y_pred)
print(f"Test MSE: {mse:.4f}")

C:\Users\Mario\AppData\Local\Temp\ipykernel_1224\3315359002.py:59: RuntimeWarning: invalid value encountered in power
  return np.power(self.left.evaluate(X), self.right.evaluate(X))
C:\Users\Mario\AppData\Local\Temp\ipykernel_1224\3315359002.py:59: RuntimeWarning: invalid value encountered in power
  return np.power(self.left.evaluate(X), self.right.evaluate(X))
C:\Users\Mario\AppData\Local\Temp\ipykernel_1224\3315359002.py:59: RuntimeWarning: invalid value encountered in power
  return np.power(self.left.evaluate(X), self.right.evaluate(X))
C:\Users\Mario\AppData\Local\Temp\ipykernel_1224\3315359002.py:59: RuntimeWarning: invalid value encountered in power
  return np.power(self.left.evaluate(X), self.right.evaluate(X))
C:\Users\Mario\AppData\Local\Temp\ipykernel_1224\3315359002.py:59: RuntimeWarning: invalid value encountered in power
  return np.power(self.left.evaluate(X), self.right.evaluate(X))
C:\Users\Mario\AppData\Local\Temp\ipykernel_1224\3315359002.py:59: RuntimeWarning: in

Invalid prediction during evaluation
Invalid prediction during evaluation
Invalid prediction during evaluation
Invalid prediction during evaluation
Invalid prediction during evaluation
Invalid prediction during evaluation
Invalid prediction during evaluation
Invalid prediction during evaluation
Invalid prediction during evaluation
Invalid prediction during evaluation
Invalid prediction during evaluation


C:\Users\Mario\AppData\Local\Temp\ipykernel_1224\3315359002.py:59: RuntimeWarning: invalid value encountered in power
  return np.power(self.left.evaluate(X), self.right.evaluate(X))
C:\Users\Mario\AppData\Local\Temp\ipykernel_1224\3315359002.py:59: RuntimeWarning: invalid value encountered in power
  return np.power(self.left.evaluate(X), self.right.evaluate(X))
C:\Users\Mario\AppData\Local\Temp\ipykernel_1224\3315359002.py:59: RuntimeWarning: invalid value encountered in power
  return np.power(self.left.evaluate(X), self.right.evaluate(X))
C:\Users\Mario\AppData\Local\Temp\ipykernel_1224\3315359002.py:59: RuntimeWarning: invalid value encountered in power
  return np.power(self.left.evaluate(X), self.right.evaluate(X))
C:\Users\Mario\AppData\Local\Temp\ipykernel_1224\3315359002.py:59: RuntimeWarning: invalid value encountered in power
  return np.power(self.left.evaluate(X), self.right.evaluate(X))


Invalid prediction during evaluation
Invalid prediction during evaluation
Invalid prediction during evaluation
Invalid prediction during evaluation
Invalid prediction during evaluation


C:\Users\Mario\AppData\Local\Temp\ipykernel_1224\3315359002.py:59: RuntimeWarning: invalid value encountered in power
  return np.power(self.left.evaluate(X), self.right.evaluate(X))
C:\Users\Mario\AppData\Local\Temp\ipykernel_1224\3315359002.py:59: RuntimeWarning: invalid value encountered in power
  return np.power(self.left.evaluate(X), self.right.evaluate(X))
C:\Users\Mario\AppData\Local\Temp\ipykernel_1224\3315359002.py:59: RuntimeWarning: invalid value encountered in power
  return np.power(self.left.evaluate(X), self.right.evaluate(X))
C:\Users\Mario\AppData\Local\Temp\ipykernel_1224\3315359002.py:59: RuntimeWarning: invalid value encountered in power
  return np.power(self.left.evaluate(X), self.right.evaluate(X))
c:\Users\Mario\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_regression.py:478: RuntimeWarning: overflow encountered in square
  output_errors = np.average((y_true - y_pred) ** 2, axis=0, weights=sample_weight)
C:\Users\Mario\AppData\Local

...

Invalid prediction during evaluation
Invalid prediction during evaluation


C:\Users\Mario\AppData\Local\Temp\ipykernel_1224\3315359002.py:59: RuntimeWarning: invalid value encountered in power
  return np.power(self.left.evaluate(X), self.right.evaluate(X))
C:\Users\Mario\AppData\Local\Temp\ipykernel_1224\3315359002.py:59: RuntimeWarning: invalid value encountered in power
  return np.power(self.left.evaluate(X), self.right.evaluate(X))


Invalid prediction during evaluation
Invalid prediction during evaluation


C:\Users\Mario\AppData\Local\Temp\ipykernel_1224\3315359002.py:59: RuntimeWarning: invalid value encountered in power
  return np.power(self.left.evaluate(X), self.right.evaluate(X))
C:\Users\Mario\AppData\Local\Temp\ipykernel_1224\3315359002.py:59: RuntimeWarning: invalid value encountered in power
  return np.power(self.left.evaluate(X), self.right.evaluate(X))


Invalid prediction during evaluation
Invalid prediction during evaluation
Generation 20, Best Fitness: 0.0368, Best Program Depth: 3
Best program (math formula): ((x1 + x2) - x3)
Test MSE: 0.0368


## Evaluation

In [19]:

import numpy as np
from gplearn.genetic import SymbolicRegressor
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
# Example dataset
np.random.seed(42)
x = np.random.uniform(-10, 10, 100).reshape(-1, 1)  # Input feature
y = 3 * x[:, 0]**2 - 2 * x[:, 0] + 5 + np.random.normal(0, 10, 100)  # Target variable
problem = np.load('problem_0.npz')
x = problem['x'].T
y = problem['y']

x.shape, y.shape

x_train, x_valid, y_train, y_valid = train_test_split(x, y, test_size=0.2, random_state=42)

# Define the symbolic regressor
est = SymbolicRegressor(
    population_size=2000,
    generations=20,
    stopping_criteria=0.01,
    p_crossover=0.7,
    p_subtree_mutation=0.1,
    p_hoist_mutation=0.05,
    p_point_mutation=0.1,
    max_samples=0.9,
    verbose=1,
    parsimony_coefficient=0.01,
    random_state=42
)

# Fit the model
est.fit(x_train, y_train)

# Predict on test data
y_pred = est.predict(x_valid)

# Print the resulting formula
print("Best formula:", est._program)


# Evaluate and visualize
mse = mean_squared_error(y_valid, y_pred)
print(f"Mean Squared Error on Test Set: {mse}")


    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0    37.95          32852.3        5        0.0924583        0.0962209      1.07m
   1    10.54          2.65077       35        0.0338287        0.0457788     34.87s
   2     5.80          1.79536        7        0.0211094        0.0195035     29.16s
   3     1.64         0.621582        7        0.0207309        0.0229098     26.99s
   4     2.10         0.510679        7        0.0208262        0.0220528     23.89s
   5     4.15         0.996103        9        0.0185058        0.0216999     23.38s
   6     5.27          1.03254        9        0.0184478        0.0222221     21.21s
   7     5.37          1.78135        9        0.0183328        0.0232565     20.30s
   8     5.31          1.01744        9        0.0184913        0.0218301  

In [ ]:
# Initialize and train
from sklearn.metrics import mean_squared_error
genP = GeneticProgramming(population_size=200, generations=100)
print(x_train.shape)
print(y_train.shape)
genP.train(x_train.T, y_train)

# Get the mathematical formula as string
print("Evolved formula:", genP.get_math_formula())

# Get the executable function
evolved_function = genP.get_function_math_formula()

# Evaluate the evolved function
print(f"MSE (train): {mean_squared_error(y_train, evolved_function(x_train))}")
print(f"MSE (real) : {mean_squared_error(y_validation, evolved_function(x_validation))}")